# Visualisation de l'écosystème normatif des webservices OGC
## Définition des fonctions nécéssaires
### <u> edge_weight :</u> Définition du poid pour un noeud donné
* **<u>Entrées :</u>**
 * **ogc_nx :** L'instance nx.DiGraph() des noeuds que l'on souhaite analyser.
 * **standard :** Standard source (Déjà présent de base dans le fichier) à l'origine de la référence (départ de l'arête).
 * **ref :** Référence présente dans la source (destination de l'arête).
 * **OGCDict :** Dictionnaire de départ comportant toutes les normes et référence.
* <u>Sortie :</u>
 * **w :** Poid de la relation entre **standard :** et **ref :**

In [10]:
def edge_weight(ogc_nx,standard,ref,OGCDict):
    #Fréquence d'apparission de la référence dans le standard (Normal de rajouter +1 car il est tout de même cité dans la partie Référence normative)
    tf_ref = OGCDict[standard]["references"][ref] + 1

    #Nombre totale de standard de base.
    N = len(OGCDict.keys())
    
    #Nombre de fois que la référence est citée dans l'écosystème
    listedge = ogc_nx.edges()
    sf_ref = 1

    for edge in listedge:
        src, to = edge
        if to==ref:
            sf_ref=sf_ref+1

    isf_ref = math.log(len(OGCDict.keys())/sf_ref)
    #w = math.log(tf_ref * isf_ref)
    w = tf_ref * isf_ref
    
    return w
    #Ajout du paramètres de poids dans la relation
#    for edge in listedge:
#        if ogc_net.edges["from"] == src and ogc_net.edges["to"] == ref :
#            key, value = "value", w
#            listedge[listedge.index(edge)].update({key: value})

### <u>rgb_to_hex :</u> Simple convertion de couleurs RGB en Hexadécimal
* **<u>Entrées :</u>**
 * **rgb :** Tuple comportant les valeurs (r, g, b)
* **<u>Sortie :</u>**
 * Valeur hexadécimale brute

In [11]:
def rgb_to_hex(rgb):
  return '%02x%02x%02x' % rgb

## Ajout des noeuds à la base de données graphs

In [12]:
from pyvis.network import Network
import json
import networkx as nx
import math

#Mise en dico du fichier json
with open("test.json", 'r') as fp:
    InitOGCDict = json.load(fp)

#Visualisation du dico
ogc_net = Network(directed=True, notebook=True, cdn_resources='in_line')
ogc_nx = nx.DiGraph()

##Alimentation du réseau
##Initialisation des noeuds avec les normes de base
ogc_nx.add_nodes_from(InitOGCDict.keys())
print("Noeuds Principaux Ajouté !")
##Ajout du reste des noeuds qui ne font pas partie des normes de base
for key in InitOGCDict.keys():
    ogc_nx.add_nodes_from(InitOGCDict[key]["references"].keys())
print("Noeuds secondaire ajouté")
##Ajout des liens entre les différents 
for src in InitOGCDict.keys():
    for dest in InitOGCDict[src]["references"].keys():
        #Ajout du lien avec le calcul de son poid
        try:
            #print('Source : ' + src)
            #print('Référence : ' + dest)
            ogc_nx.add_edge(src,dest,value=edge_weight(ogc_nx,src,dest,InitOGCDict)) #Peut être voir pour mettre value à la place si pyvis comprend mieux
        except KeyError as e:
            print(e)
            #print("La relation " + src + " -> " + dest + " n'a pas pu être calculé")
        #Réglage de la taille des noeuds
        ogc_nx.nodes[src]['size']=len(InitOGCDict[src]['references'].keys())*3
        
#ogc_net.from_nx(ogc_nx)
#ogc_net.force_atlas_2based()
#ogc_net.prep_notebook()
#ogc_net.show("results/ogc_ws.html")

Noeuds Principaux Ajouté !
Noeuds secondaire ajouté


## Analyse par degré de centralité avec changement de couleurs des normes de base
* Couleur des noeuds :
 * Nuance de Rouge = référence
 * Vert/Blanc = norme de l'écosystème analysé

In [13]:
nx.degree_centrality(ogc_nx)
degree_centrality = nx.in_degree_centrality(ogc_nx)
max_key = max(degree_centrality, key=lambda key: degree_centrality[key])
max_degree = degree_centrality[max_key]
Verifdic=dict()
srclist=list()

for n in ogc_nx.nodes:
    if n in InitOGCDict.keys() :
        srclist.append(src)
        Verifdic["vert"] = srclist
        r=255*abs(((degree_centrality[n])/(max_degree)+0.1)-1)
        g=255*abs(((degree_centrality[n])/(max_degree)+0.1)-1)
        b=255
        rgb = (int(r),int(b),int(g))
        ogc_nx.nodes[n]['color']="#"+rgb_to_hex(rgb)
    else :
        srclist.append(src)
        Verifdic["rouge"] = srclist
        r=255
        g=255*abs((degree_centrality[n]/max_degree)-1)
        b=255*abs((degree_centrality[n]/max_degree)-1)
        rgb = (int(r),int(b),int(g))
        ogc_nx.nodes[n]['color']="#"+rgb_to_hex(rgb)

## Visualisation

In [14]:
ogc_net.from_nx(ogc_nx)
ogc_net.inherit_edge_colors(False)
#ogc_net.force_atlas_2based()
ogc_net.prep_notebook()
ogc_net.show("results/ogc_ws_centrality_dir.html")

results/ogc_ws_centrality_dir.html


### Résultat de voterank

In [9]:
nx.voterank(ogc_nx)

['OGC 06-121r9',
 'OGC 19-087',
 'OGC 19-086r5',
 'OGC 18-062r2',
 'OGC 09-025r2',
 'OGC 15-001r4',
 'OGC 20-057',
 'OGC 08-068r2',
 'OGC 09-083r4',
 'OGC 19-072',
 'OGC 17-089r1',
 'OGC 06-042',
 'OGC 17-069r4',
 'OGC 05-005',
 'OGC 11-014r3',
 'OGC 07-057r7',
 'OGC 14-065',
 'OGC 20-024']

## Récupération du poid des arêtes

In [8]:
print(ogc_nx.edges.data('value'))

[('OGC 05-005', 'IETF RFC 2045', 2.8903717578961645), ('OGC 05-005', 'IETF RFC 2119', 5.780743515792329), ('OGC 05-005', 'SLD', 11.561487031584658), ('OGC 05-005', 'XML 1.0', 8.671115273688493), ('OGC 06-042', 'ISO 8601:2004', 20.23260230527315), ('OGC 06-042', 'ISO 19111', 23.122974063169316), ('OGC 06-042', 'ISO 19115', 17.342230547376985), ('OGC 06-042', 'IETF RFC 2045', 4.394449154672439), ('OGC 06-042', 'IETF RFC 2396', 17.342230547376985), ('OGC 06-042', 'IETF RFC 2616', 28.903717578961647), ('OGC 06-042', 'UCUM', 8.671115273688493), ('OGC 06-042', 'XML 1.0', 6.591673732008658), ('OGC 06-042', 'XML Schema', 11.561487031584658), ('OGC 06-121r9', 'CGI', 2.8903717578961645), ('OGC 06-121r9', 'IETF RFC 2045', 3.58351893845611), ('OGC 06-121r9', 'IETF RFC 2141', 2.8903717578961645), ('OGC 06-121r9', 'IETF RFC 2396', 15.380572041353537), ('OGC 06-121r9', 'IETF RFC 2616', 19.775021196025975), ('OGC 06-121r9', 'IETF RFC 4646', 14.451858789480823), ('OGC 06-121r9', 'ISO/IEC Directives, Pa

In [9]:
ogc_nx['OGC 05-005']['HTML5']['value']

KeyError: 'HTML5'

In [10]:
G = ogc_nx
G['OGC 05-005']['IETF RFC 2045']['value']

2.8903717578961645

In [11]:
for src in InitOGCDict.keys():
        rez=0
        nb_dep = 0
        for dest in ogc_nx.nodes():
            try :
                rez=rez+G[src][dest]['value']
                nb_dep = nb_dep + 1
            except KeyError :
                None
        print(str(src) + " à pour exposition : " + str(rez) + " pour " + str(nb_dep) + " N-1 dépendences.")

OGC 05-005 à pour exposition : 28.903717578961643 pour 4 dépendences.
OGC 06-042 à pour exposition : 138.16248023411234 pour 9 dépendences.
OGC 06-121r9 à pour exposition : 355.25514621553305 pour 26 dépendences.
OGC 07-057r7 à pour exposition : 240.13642197669444 pour 4 dépendences.
OGC 08-068r2 à pour exposition : 99.08356998468592 pour 9 dépendences.
OGC 09-025r2 à pour exposition : 417.3121454257158 pour 12 dépendences.
OGC 09-083r4 à pour exposition : 199.21363625690094 pour 6 dépendences.
OGC 11-014r3 à pour exposition : 42.44041414994811 pour 3 dépendences.
OGC 14-065 à pour exposition : 145.8006502536501 pour 6 dépendences.
OGC 15-001r4 à pour exposition : 64.58917001882008 pour 9 dépendences.
OGC 17-089r1 à pour exposition : 56.53892383245978 pour 6 dépendences.
OGC 19-072 à pour exposition : 131.04755835833913 pour 9 dépendences.
OGC 20-024 à pour exposition : 118.8585911807119 pour 8 dépendences.
OGC 19-086r5 à pour exposition : 285.47850189115746 pour 16 dépendences.
OGC 17

In [28]:
print(degree_centrality)

{'OGC 05-005': 0.0, 'OGC 06-042': 0.0, 'OGC 06-121r9': 0.022988505747126436, 'OGC 07-057r7': 0.0, 'OGC 08-068r2': 0.0, 'OGC 09-025r2': 0.0, 'OGC 09-083r4': 0.0, 'OGC 11-014r3': 0.0, 'OGC 14-065': 0.0, 'OGC 15-001r4': 0.0, 'OGC 17-089r1': 0.0, 'OGC 19-072': 0.011494252873563218, 'OGC 20-024': 0.0, 'IETF RFC 2045': 0.034482758620689655, 'IETF RFC 2119': 0.011494252873563218, 'SLD': 0.011494252873563218, 'XML 1.0': 0.05747126436781609, 'ISO 8601:2004': 0.022988505747126436, 'ISO 19111': 0.022988505747126436, 'ISO 19115': 0.034482758620689655, 'IETF RFC 2396': 0.022988505747126436, 'IETF RFC 2616': 0.04597701149425287, 'UCUM': 0.011494252873563218, 'XML Schema': 0.04597701149425287, 'CGI': 0.011494252873563218, 'IETF RFC 2141': 0.011494252873563218, 'IETF RFC 4646': 0.034482758620689655, 'ISO/IEC Directives, Part 2': 0.011494252873563218, 'ISO 4217:2001': 0.011494252873563218, 'ISO 8601:2000': 0.011494252873563218, 'ISO 19119': 0.011494252873563218, 'ISO 19123': 0.011494252873563218, 'OGC 